In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/veremi-29f-6classes/y.csv
/kaggle/input/veremi-29f-6classes/X.csv


In [7]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
import time
import os
import sys
import joblib
from datetime import datetime

def print_flush(*args, **kwargs):
    print(*args, **kwargs)
    sys.stdout.flush()

# ========== CONFIGURATION ==========
print_flush("Starting Sender-Aware XGBoost model training...")

# ========== GPU-ENABLED XGBOOST PARAMETERS ==========
BEST_XGB_PARAMS = {
    'objective': 'multi:softmax',
    'num_class': 6,
    'eval_metric': 'mlogloss',
    'max_depth': 8,
    'learning_rate': 0.068,
    'n_estimators': 501,
    'subsample': 0.92,
    'colsample_bytree': 0.80,
    'random_state': 42,
    'verbosity': 1,
    'n_jobs': -1,
    'tree_method': 'gpu_hist'
}

# ========== LOAD DATA ==========
print_flush("Loading data from Kaggle input...")
X_full = pd.read_csv('/kaggle/input/veremi-29f-6classes/X.csv')
y_full = pd.read_csv('/kaggle/input/veremi-29f-6classes/y.csv')

print_flush(f"Original X shape: {X_full.shape}")
print_flush(f"Original y shape: {y_full.shape}")

# Keep only the 15 required features
REQUIRED_FEATURES = ['sendtime', 'senderpseudo', 'posx', 'posy', 'posx_n', 
                    'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 
                    'hedx', 'hedy', 'hedx_n', 'hedy_n']

print_flush("Filtering to 15 required features...")
missing_features = [f for f in REQUIRED_FEATURES if f not in X_full.columns]
if missing_features:
    raise ValueError(f"Missing required features: {missing_features}")

X_filtered = X_full[REQUIRED_FEATURES].copy()
print_flush(f"Filtered X shape: {X_filtered.shape}")

# Extract y values (assuming single column)
y_values = y_full.iloc[:, 0].values

# ========== SENDER-AWARE SPLITTING ==========
def create_sender_aware_splits(X, y, train_val_ratio=0.80, random_state=42):
    print_flush("\n========== Sender-Aware Data Splitting ==========")
    senders = X['senderpseudo'].values
    unique_senders = np.unique(senders)
    print_flush(f"Total unique senders: {len(unique_senders)}")
    
    np.random.seed(random_state)
    n_train_val_senders = int(train_val_ratio * len(unique_senders))
    shuffled_senders = np.random.permutation(unique_senders)
    train_val_senders = shuffled_senders[:n_train_val_senders]
    test_senders = shuffled_senders[n_train_val_senders:]
    
    print_flush(f"Train/Val senders: {len(train_val_senders)} ({len(train_val_senders)/len(unique_senders)*100:.1f}%)")
    print_flush(f"Test senders: {len(test_senders)} ({len(test_senders)/len(unique_senders)*100:.1f}%)")
    
    train_val_mask = np.isin(senders, train_val_senders)
    test_mask = np.isin(senders, test_senders)
    
    X_train_val = X[train_val_mask].reset_index(drop=True)
    y_train_val = y[train_val_mask]
    X_test_new = X[test_mask].reset_index(drop=True)
    y_test_new = y[test_mask]
    
    # Verify no sender overlap
    train_val_sender_set = set(X_train_val['senderpseudo'].unique())
    test_sender_set = set(X_test_new['senderpseudo'].unique())
    overlap = len(train_val_sender_set & test_sender_set)
    print_flush(f"Sender overlap: {overlap} (should be 0)")
    
    return X_train_val, y_train_val, X_test_new, y_test_new

# Perform sender-aware split
X_train_val, y_train_val, X_test_unseen, y_test_unseen = create_sender_aware_splits(
    X_filtered, y_values, train_val_ratio=0.80
)

# ========== STANDARDIZATION ==========
print_flush("\nApplying standardization to numerical features...")
numerical_features = ['posx', 'posy', 'posx_n', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 
                     'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']

scaler_initial = StandardScaler()
X_train_val_scaled = X_train_val.copy()
X_test_scaled = X_test_unseen.copy()

X_train_val_scaled[numerical_features] = scaler_initial.fit_transform(X_train_val[numerical_features])
X_test_scaled[numerical_features] = scaler_initial.transform(X_test_unseen[numerical_features])

# ========== FEATURE SELECTION ==========
FEATURES = ['senderpseudo', 'posx', 'posy', 'posx_n', 'spdx', 'spdy', 'hedy', 'hedx_n']

def check_and_select_features(df: pd.DataFrame, features: list, df_name: str):
    missing = [f for f in features if f not in df.columns]
    if missing:
        raise ValueError(f"Missing features in {df_name}: {missing}")
    return df[features].copy()

print_flush(f"\nSelecting 8 core features: {FEATURES}")
X_train_val_selected = check_and_select_features(X_train_val_scaled, FEATURES, 'X_train_val')
X_test_selected = check_and_select_features(X_test_scaled, FEATURES, 'X_test')

# ========== FEATURE ENGINEERING ==========
def engineer_advanced_features(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    print_flush("Starting advanced feature engineering...")
    
    # Sequence and timing features
    X['sender_sequence'] = X.groupby('senderpseudo').cumcount()
    X['inter_arrival_time'] = X.groupby('senderpseudo')['sender_sequence'].diff().fillna(1.0)
    
    # Speed features
    X['speed_magnitude'] = np.sqrt(X['spdx']**2 + X['spdy']**2)
    X['acceleration_x'] = X.groupby('senderpseudo')['spdx'].diff().fillna(0)
    X['acceleration_y'] = X.groupby('senderpseudo')['spdy'].diff().fillna(0)
    X['acceleration_magnitude'] = np.sqrt(X['acceleration_x']**2 + X['acceleration_y']**2)
    
    # Position features
    X['position_change_x'] = X.groupby('senderpseudo')['posx'].diff().fillna(0)
    X['position_change_y'] = X.groupby('senderpseudo')['posy'].diff().fillna(0)
    X['position_change_magnitude'] = np.sqrt(X['position_change_x']**2 + X['position_change_y']**2)
    
    # Heading features
    X['heading_change'] = X.groupby('senderpseudo')['hedy'].diff().fillna(0)
    X['heading_magnitude'] = np.sqrt(X['hedx_n']**2 + X['hedy']**2)
    
    # Consistency features
    X['speed_consistency'] = X.groupby('senderpseudo')['speed_magnitude'].transform('std').fillna(0)
    X['position_consistency'] = X.groupby('senderpseudo')['position_change_magnitude'].transform('std').fillna(0)
    
    # Time-based features
    X['hour'] = (X['sender_sequence'] % 24)
    X['night_hours'] = X['hour'].between(22, 5, inclusive="left").astype(int)
    
    # Interaction features
    X['speed_position_interaction'] = X['speed_magnitude'] * X['position_change_magnitude']
    X['inter_arrival_speed_ratio'] = X['inter_arrival_time'] / (X['speed_magnitude'] + 1e-6)
    
    # Fill any remaining NaN values
    X = X.fillna(0)
    print_flush(f"Feature engineering completed. New shape: {X.shape}")
    return X

print_flush("\n========== Enhanced Feature Engineering ==========")
X_train_val_engineered = engineer_advanced_features(X_train_val_selected)
X_test_engineered = engineer_advanced_features(X_test_selected)

FEATURES_ENGINEERED = list(X_train_val_engineered.columns)
print_flush(f"Total features after engineering: {len(FEATURES_ENGINEERED)}")

# ========== MODEL TRAINING ==========
def run_sender_aware_xgboost():
    print_flush("\n========== Running Sender-Aware XGBoost Model (GPU) ==========")
    print_flush(f"Using {len(FEATURES_ENGINEERED)} engineered features")
    print_flush("🎯 Sender-aware validation: NO SENDER LEAKAGE")
    
    # Class distribution analysis
    print_flush("Class distribution in train/val set:")
    unique, counts = np.unique(y_train_val, return_counts=True)
    for cls, count in zip(unique, counts):
        print_flush(f"Class {cls}: {count} samples ({count/len(y_train_val)*100:.2f}%)")

    # Calculate class weights
    class_counts = np.bincount(y_train_val)
    total_samples = len(y_train_val)
    class_weights = total_samples / (len(class_counts) * class_counts)
    print_flush("Class weights:", {i: weight for i, weight in enumerate(class_weights)})

    # Set up cross-validation with k=5
    k = 5
    group_kfold = GroupKFold(n_splits=k)
    groups = X_train_val_engineered['senderpseudo'].values

    fold_accuracies = []
    fold_f1_scores = []

    print_flush(f"\nStarting {k}-Fold Sender-Aware Cross Validation (GPU)...")

    params = BEST_XGB_PARAMS.copy()

    # Final standardization for engineered features
    scaler = StandardScaler()
    X_train_val_final = scaler.fit_transform(X_train_val_engineered)
    X_test_final = scaler.transform(X_test_engineered)

    for fold, (train_index, val_index) in enumerate(group_kfold.split(X_train_val_final, y_train_val, groups=groups)):
        print_flush(f"\n--- Fold {fold+1}/{k} (Sender-Aware) ---")
        
        X_train_fold = X_train_val_final[train_index]
        X_val_fold = X_train_val_final[val_index]
        y_train_fold = y_train_val[train_index]
        y_val_fold = y_train_val[val_index]
        
        # Verify sender separation
        train_senders = set(X_train_val_engineered.iloc[train_index]['senderpseudo'].unique())
        val_senders = set(X_train_val_engineered.iloc[val_index]['senderpseudo'].unique())
        overlap = len(train_senders & val_senders)
        
        print_flush(f"Train fold size: {X_train_fold.shape[0]} ({len(train_senders)} senders)")
        print_flush(f"Val fold size: {X_val_fold.shape[0]} ({len(val_senders)} senders)")
        print_flush(f"Sender overlap: {overlap} (should be 0)")
        
        # Train model
        model_fold = XGBClassifier(**params)
        sample_weights = np.array([class_weights[y] for y in y_train_fold])
        
        start_time = time.time()
        model_fold.fit(X_train_fold, y_train_fold, sample_weight=sample_weights)
        training_time = time.time() - start_time
        
        print_flush(f"Fold {fold+1} training time: {training_time:.2f} seconds")
        
        # Evaluate
        y_pred_fold = model_fold.predict(X_val_fold)
        fold_accuracy = accuracy_score(y_val_fold, y_pred_fold)
        fold_f1 = f1_score(y_val_fold, y_pred_fold, average='macro')
        
        fold_accuracies.append(fold_accuracy)
        fold_f1_scores.append(fold_f1)
        
        print_flush(f"Fold {fold+1} Validation Accuracy: {fold_accuracy:.4f}")
        print_flush(f"Fold {fold+1} Validation F1-Score: {fold_f1:.4f}")
        
        print_flush(f"\nFold {fold+1} Classification Report:")
        print_flush(classification_report(y_val_fold, y_pred_fold))

    # Test final model performance
    print_flush("\n===== Training Final Model on All Training Data =====")
    final_model = XGBClassifier(**params)
    final_model.fit(X_train_val_final, y_train_val,
                   sample_weight=np.array([class_weights[y] for y in y_train_val]))
    
    # Test on unseen senders
    y_test_pred = final_model.predict(X_test_final)
    test_accuracy = accuracy_score(y_test_unseen, y_test_pred)
    test_f1_macro = f1_score(y_test_unseen, y_test_pred, average='macro')
    test_f1_weighted = f1_score(y_test_unseen, y_test_pred, average='weighted')
    
    print_flush(f"\n===== Sender-Aware XGBoost Performance =====")
    print_flush(f"Validation (CV) Accuracy: {np.mean(fold_accuracies):.4f} (+/- {np.std(fold_accuracies)*2:.4f})")
    print_flush(f"Validation (CV) F1-Macro: {np.mean(fold_f1_scores):.4f} (+/- {np.std(fold_f1_scores)*2:.4f})")
    print_flush(f"Test (Unseen Senders) Accuracy: {test_accuracy:.4f}")
    print_flush(f"Test (Unseen Senders) F1-Macro: {test_f1_macro:.4f}")
    print_flush(f"Test (Unseen Senders) F1-Weighted: {test_f1_weighted:.4f}")

    # Overfitting analysis
    cv_test_gap = np.mean(fold_accuracies) - test_accuracy
    print_flush(f"\n========== Overfitting Analysis ==========")
    print_flush(f"CV-Test Accuracy Gap: {cv_test_gap:.4f} ({cv_test_gap*100:.2f}%)")
    
    if cv_test_gap < 0.02:
        print_flush("✅ Excellent generalization - minimal overfitting")
    elif cv_test_gap < 0.05:
        print_flush("✅ Good generalization - acceptable overfitting")
    elif cv_test_gap < 0.10:
        print_flush("⚠️ Moderate overfitting - consider regularization")
    else:
        print_flush("❌ Severe overfitting - model memorizing sender patterns")
    
    print_flush("\nTest Set (Unseen Senders) Classification Report:")
    print_flush(classification_report(y_test_unseen, y_test_pred))

    # ========== SAVE ONLY THE TWO REQUIRED FILES ==========
    print_flush("\n===== Saving Final Model and Scaler for Website =====")
    joblib.dump(final_model, 'final_xgboost_model.pkl')
    joblib.dump(scaler, 'final_xgboost_scaler.pkl')
    
    print_flush("✅ Successfully saved:")
    print_flush("   - final_xgboost_model.pkl")
    print_flush("   - final_xgboost_scaler.pkl")
    print_flush("\n🚀 These files are ready for your website deployment!")

    return final_model, scaler

if __name__ == "__main__":
    print_flush("Starting Sender-Aware XGBoost 6-Class model training...")
    print_flush("🎯 TRUE SENDER-AWARE VALIDATION - NO SENDER LEAKAGE")
    print_flush("✅ 80% senders for train/val, 20% senders for test")
    print_flush("✅ GroupKFold with k=5 ensures no sender appears in both train and val")
    
    final_model, scaler = run_sender_aware_xgboost()
    
    print_flush("\n🎉 Training completed successfully!")
    print_flush("📦 Only the required files have been saved for website deployment")


Starting Sender-Aware XGBoost model training...
Loading data from Kaggle input...
Original X shape: (3194808, 29)
Original y shape: (3194808, 1)
Filtering to 15 required features...
Filtered X shape: (3194808, 15)

========== Sender-Aware Data Splitting ==========
Total unique senders: 118909
Train/Val senders: 95127 (80.0%)
Test senders: 23782 (20.0%)
Sender overlap: 0 (should be 0)

Applying standardization to numerical features...

Selecting 8 core features: ['senderpseudo', 'posx', 'posy', 'posx_n', 'spdx', 'spdy', 'hedy', 'hedx_n']

========== Enhanced Feature Engineering ==========
Starting advanced feature engineering...
Feature engineering completed. New shape: (2560342, 25)
Starting advanced feature engineering...
Feature engineering completed. New shape: (634466, 25)
Total features after engineering: 25
Starting Sender-Aware XGBoost 6-Class model training...
🎯 TRUE SENDER-AWARE VALIDATION - NO SENDER LEAKAGE
✅ 80% senders for train/val, 20% senders for test
✅ GroupKFold with 

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [09:29:01] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Fold 1 training time: 63.04 seconds


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [09:30:01] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [09:30:01] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Fold 1 Validation Accuracy: 0.9474
Fold 1 Validation F1-Score: 0.8910

Fold 1 Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98    300198
           1       0.97      0.86      0.91     32312
           2       0.87      0.90      0.89     44596
           3       0.86      0.90      0.88     20478
           4       0.85      0.70      0.77      9336
           5       0.95      0.89      0.92    105149

    accuracy                           0.95    512069
   macro avg       0.91      0.87      0.89    512069
weighted avg       0.95      0.95      0.95    512069


--- Fold 2/5 (Sender-Aware) ---
Train fold size: 2048273 (76087 senders)
Val fold size: 512069 (19040 senders)
Sender overlap: 0 (should be 0)


...

Fold 5 training time: 62.88 seconds


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [09:34:23] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Fold 5 Validation Accuracy: 0.9454
Fold 5 Validation F1-Score: 0.9061

Fold 5 Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98    307240
           1       0.96      0.84      0.90     27802
           2       0.83      0.91      0.87     36699
           3       0.88      0.90      0.89     21828
           4       0.97      0.84      0.90     28203
           5       0.95      0.86      0.90     90296

    accuracy                           0.95    512068
   macro avg       0.93      0.89      0.91    512068
weighted avg       0.95      0.95      0.94    512068


===== Training Final Model on All Training Data =====


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [09:34:29] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [09:35:41] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)



===== Sender-Aware XGBoost Performance =====
Validation (CV) Accuracy: 0.9411 (+/- 0.0090)
Validation (CV) F1-Macro: 0.8953 (+/- 0.0129)
Test (Unseen Senders) Accuracy: 0.9417
Test (Unseen Senders) F1-Macro: 0.8987
Test (Unseen Senders) F1-Weighted: 0.9410

========== Overfitting Analysis ==========
CV-Test Accuracy Gap: -0.0006 (-0.06%)
✅ Excellent generalization - minimal overfitting

Test Set (Unseen Senders) Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98    384093
           1       0.95      0.83      0.89     32365
           2       0.80      0.92      0.85     48489
           3       0.86      0.91      0.89     27294
           4       0.92      0.88      0.90     27675
           5       0.96      0.82      0.89    114550

    accuracy                           0.94    634466
   macro avg       0.91      0.89      0.90    634466
weighted avg       0.94      0.94      0.94    634466


===== Saving Fin